Rokas Petrauskas 1gr. 2pogr.

In [172]:
import matplotlib.pyplot as plt # for plotting
import numpy as np # for transformation

import torch # PyTorch package
import torchvision # load datasets
import torchvision.transforms as transforms # transform data
import torch.nn as nn # basic building block for neural networks
import torch.nn.functional as F # import convolution functions like Relu
import torch.optim as optim # optimzer
import torch.utils.data as data
import torchvision.datasets as datasets

In [173]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Dataset parsiuntimas. uzkomentuotas kas leidziant viska nepasikartotu.

In [174]:
# from openimages.download import download_dataset
# download_dataset (dest_dir = "C:/Users/Rokas/Desktop/univer/gmm/lab2/test_data", class_labels = ["Airplane", "Boat", "Car"], csv_dir = "C:/Users/Rokas/Desktop/univer/gmm/lab2/data", limit = 300)

Duomenu transformacijos

In [175]:
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.3, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(20),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

load train data

In [176]:
train_dir = 'C:/Users/Rokas/Desktop/univer/gmm/lab2/data/'
train_dataset = datasets.ImageFolder(train_dir, transform = transform_train)
batch_size = 8
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

load test data

In [177]:
test_dir = 'C:/Users/Rokas/Desktop/univer/gmm/lab2/test_data'
test_dataset = datasets.ImageFolder(test_dir, transform = transform_test)
batch_size = 8
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

klasiu aprasymas

In [178]:
classes = ('airplane', 'boat', 'car')

apsirasom architektura

In [179]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.relu3 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(128, 3)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 32 * 56 * 56)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        return x

instantiate the cnn

In [180]:
net = Net()
net = Net().to(device)

nuostoliu funkcija ir optimizacija

In [181]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-4, momentum=0.9)

training the model

In [182]:
for epoch in range(15):  # epochs

    running_loss = 0.0
    for i, data in enumerate(train_data_loader, 0):
        net.train()
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # # print statistics
        # running_loss += loss.item()
        # if i % 2000 == 1999:    # print every 2000 mini-batches
        #     print('[%d, %5d] loss: %.3f' %
        #           (epoch + 1, i + 1, running_loss / 2000))
        #     running_loss = 0.0

issaugom modeli / loadinam modeli

In [183]:
# # save
# PATH = 'pirmas_modelis/LOCAL01.pth'
# torch.save(net.state_dict(), PATH)
# # reload
# net = Net()
# net.load_state_dict(torch.load(PATH))

testing the model

In [184]:
net.eval()
y_true = []
y_pred = []

for inputs, labels in test_data_loader:
    inputs, labels = inputs.to(device), labels.to(device)

    with torch.no_grad():
        outputs = net(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

    y_true.extend(labels.cpu().numpy())
    y_pred.extend(preds.cpu().numpy())

calculating metrics

In [185]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
import pandas as pd

# Calculate metrics
conf_matrix = confusion_matrix(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

class_names = ['airplane', 'boat', 'car']
conf_matrix_df = pd.DataFrame(conf_matrix, columns=class_names, index=class_names)

print("Confusion Matrix:")
print(conf_matrix_df)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score:: {f1_score:.4f}')

Confusion Matrix:
          airplane  boat  car
airplane       147    95   58
boat            19   213   68
car              7    59  234
Accuracy: 0.6600
Precision: 0.6934
Recall: 0.6600
F1 Score:: 0.6564
